# Demo 3: Gemini Exercise

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ciri/persona-workshop/blob/main/demo-3/Silicon-Gemini-Exercise.ipynb)

We will start by setting-up the notebook. If you haven't already, first create a Gemini API key [here](https://www.google.com/url?q=https%3A%2F%2Faistudio.google.com%2Fapp%2Fapikey) (free). The free version is somewhat limited (see quotas [here](https://cloud.google.com/gemini/docs/quotas#daily)), but if you add your card information you get $300 free credit for the next 90 days (you don't need to do this for the workshop). You can then add it below.

In [ ]:
!pip install dotenv
!pip install -U google-generativeai

In [3]:
# Libraries that we will use, if you are missing a library,
# create a new cell with e.g.:
# !pip install NAME_OF_MISSING
# where NAME_OF_MISSING is the library that you are missing.
import typing
import google.generativeai as genai
from tqdm import tqdm
import numpy as np
import pandas as pd
import typing_extensions as typing
import json
import random
import seaborn as sns
import matplotlib.pyplot as plt

## Exercise 1: Initialize the Generative Model and conduct a basic promt

Let's start by veryifying that we can initialize and call a model. Ask it to write a poem about your country of origin.

In [6]:
import google.generativeai as genai

genai.configure(api_key="Enter you API Key here")  # Enter your API key here


In [ ]:
# Use a supported model.
# model = genai.GenerativeModel(model_name="gemini-2.0-flash")
model = genai.GenerativeModel(model_name="gemini-2.5-flash-preview-05-20")
response = model.generate_content("Escribe un poema corto sobre comida de Cuernacava, Morelos Mexico")
print(response.text)

En Cuernavaca, la eterna primavera,
El sabor se viste de luz y de quimera.
La mesa llama, un festín morelense,
Con aromas que al alma hacen florecer.

De Yecapixtla, la cecina, fina y salada,
Con crema y queso, una delicia dorada.
Un bocado de sol, tendido en la tortilla,
Que en cada fibra de Morelos brilla.

Y los tacos acorazados, ¡qué invención!
Con arroz y guiso, una explosión.
De la calle el bullicio, sabor popular,
Una fortaleza rica, para el hambre saciar.

El mole verde, fresco, de pipián y hierbas,
Recuerda el campo, sus verdes reservas.
Las frutas dulces, con su néctar tan grato,
Un festín tropical, del árbol al plato.

Cuernavaca, en tu mesa hay un cantar,
De Morelos el alma, lista para gustar.


### **Here we have different options with different models**

In [ ]:
import google.generativeai as genai

# For Gemini 1.5 Pro
# model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Alternative model 2
# model = genai.GenerativeModel(model_name="gemini-2.5-flash-preview-05-20")

# Alternative model 3
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

response = model.generate_content("Escribe un poema corto sobre comida mexicana")
print(response.text)

En tierras del sol ardiente y sabor vibrante,
la cocina mexicana, un festín constante.
Maíz en tortillas, suaves al tacto,
rellenas de carne, un suculento pacto.

Chiles que arden, con fuego en la boca,
salsas que bailan, al ritmo de la toca.
Guacamole cremoso, verde y frescor,
un manjar divino, un eterno amor.

Tamales en hoja, un regalo escondido,
mole misterioso, de sabor exquisito.
De México lindo, su rica sazón,
un abrazo cálido, en cada bocado y razón.



We will now be using the [Gemini API](https://ai.google.dev/api?lang=python) to synthetic responses.

## Exercise 4: Structured output
You can ask a model to return structured output which makes it easier to post-process into statistics.

1. Specify the output format
2. Specify the input prompt template interested in.
3. Specify a polulation - for now just a list
4. Run the promp

In [7]:
# Step 1: Specify the output format
class MovieSpecification(typing.TypedDict):
    age: int
    location: str
    food: str
    profession: str

# Step 2: Specify the input prompt template
template = "You are a {age}-year old {gender} from {location} and working as {profession}."

# Step 3: specify a distribution - for now just a list.
population = [
    {"age":20, "gender":"female","location":"Thailand","profession":"student"},
    {"age":20, "gender":"male","location":"Mexico","profession":"cashier"},
    {"age":60, "gender":"female","location":"Spain","profession":"businessman"},
    # TODO: add a couple more profiles here, by copy-pasting
    #       and then modifying the line
    # ...
]

# Step 4: run the survey
for person in population:
  system_prompt = template.format(**person)
  model = genai.GenerativeModel('gemini-2.5-flash-preview-05-20', system_instruction=system_prompt)
  #model = genai.GenerativeModel('gemini-2.0-flash', system_instruction=system_prompt)
  response = model.generate_content(
      """
      Cuál es tu comida favorita para la cena? Evita responder con un estereotipo de comida
      y menciona una comida de la cocina de tu localidad que disfrutas mucho.
      """,
      generation_config = genai.GenerationConfig(
          response_mime_type="application/json", response_schema=list[MovieSpecification]
      ),
  )

  print(system_prompt)
  print(json.loads(response.text))

You are a 20-year old female from Thailand and working as student.
[{'age': 20, 'food': 'Pad Krapow Moo Saap', 'location': 'Thailand', 'profession': 'student'}]
You are a 20-year old male from Mexico and working as cashier.
[{'age': 20, 'food': 'Sopa de tortilla', 'location': 'Mexico', 'profession': 'cashier'}]
You are a 60-year old female from Spain and working as businessman.
[{'age': 60, 'food': 'Bacalao al pil-pil', 'location': 'Spain', 'profession': 'businessman'}]


## Exercise 5: Encuesta Nacional Seguridad Urbana (ENSU)

In the following code.

1. We will randomly select "n" individual responses
2. We will pass demographics features to the AI model
3. Wi will generate synthetic responses based on real demographics
4. Compare real results with synthetic results

In [8]:
####  function for maping the column SEXO into textual values
import pandas as pd

def map_sexo_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Maps numeric SEXO values to categorical labels.

    Parameters:
        df (pd.DataFrame): A DataFrame containing the column 'SEXO'

    Returns:
        pd.DataFrame: A copy of the DataFrame with 'SEXO' values mapped
    """
    sexo_map = {
        1: "Hombre",
        2: "Mujer"
    }
    df = df.copy()  # Avoid modifying the original DataFrame
    df["SEXO"] = df["SEXO"].map(sexo_map)
    return df

In [11]:
import pandas as pd

# Load the CSV file
ensu = pd.read_csv("https://www.dropbox.com/scl/fi/nsopocoldxc4xo0jwxt2a/ENSU_Dic_2024.csv?rlkey=1ss4nl2j612xjooax10ldc9lr&dl=1")

ensu = map_sexo_column(ensu)

### **Conduct a descriptive data analysis on the variables of interest**

In [16]:
# Subset the relevant columns
selected_columns = ["SEXO", "EDAD", "BP1_1", "BP1_2_01", "BP1_2_02", "BP1_2_03", "BP1_2_04"]
df = ensu[selected_columns]

# Descriptive statistics for numerical columns
numeric_stats = df.describe()

# Frequency counts for categorical columns
categorical_stats = {}
for col in ["SEXO", "BP1_1", "BP1_2_01", "BP1_2_02", "BP1_2_03", "BP1_2_04"]:
    categorical_stats[col] = df[col].value_counts(dropna=False)

# Print results
print("=== Numeric Descriptive Statistics ===")
print(numeric_stats)

print("\n=== Categorical Frequency Counts ===")
for col, counts in categorical_stats.items():
    print(f"\n-- {col} --")
    print(counts)


=== Numeric Descriptive Statistics ===
               EDAD         BP1_1      BP1_2_01      BP1_2_02      BP1_2_03  \
count  23451.000000  23451.000000  23451.000000  23451.000000  23451.000000   
mean      46.380922      1.608673      1.179779      1.772803      1.530852   
std       17.286217      0.625329      0.419668      0.888627      0.574008   
min       18.000000      1.000000      1.000000      1.000000      1.000000   
25%       32.000000      1.000000      1.000000      1.000000      1.000000   
50%       45.000000      2.000000      1.000000      1.000000      2.000000   
75%       59.000000      2.000000      1.000000      3.000000      2.000000   
max       98.000000      9.000000      9.000000      9.000000      9.000000   

           BP1_2_04  
count  23451.000000  
mean       2.925845  
std        0.690232  
min        1.000000  
25%        3.000000  
50%        3.000000  
75%        3.000000  
max        9.000000  

=== Categorical Frequency Counts ===

-- SEXO --
S

### **Randomly selecting n observations for the ENSU survey**

In [23]:
# Randomly select n rows
ensu_sample = ensu.sample(n=10, random_state=896)  # Set random_state for reproducibility

In [24]:
ensu_sample[["NOM_MUN", "SEXO", "EDAD","BP1_1"]]

,NOM_MUN,SEXO,EDAD,BP1_1
9753,ACAPULCO DE JUAREZ,Mujer,76,1
42,AGUASCALIENTES,Hombre,39,2
133,AGUASCALIENTES,Hombre,19,2
3563,TUXTLA GUTIERREZ,Hombre,42,2
17995,QUERETARO,Hombre,53,2
2940,MANZANILLO,Mujer,51,1
3845,CHIHUAHUA,Mujer,58,1
23333,GUADALUPE,Hombre,74,2
22663,MERIDA,Hombre,59,2
9108,LEON,Mujer,20,2


In [25]:
###  Example 1.  We are using short prompt
import pandas as pd
import json
from typing_extensions import TypedDict
import google.generativeai as genai

# 1. Assume ensu_sample is already loaded as a pandas DataFrame.

# Select only the variables city, sex, and age
ensu_sel = ensu_sample[["NOM_MUN", "SEXO", "EDAD"]]

# 2. Define the survey question template
survey_questions = {
    "BP1_1": "En términos de delincuencia, ¿considera que vivir en {NOM_MUN} es…"
}

#3. Define the response mapping dictionary
BP1_dic = {
    1: "Seguro",
    2: "Inseguro",
    3: "No sabe / no responde"
}

# 4. Define expected JSON output schema
class SurveyResponse(TypedDict):
    response_code: int

# 5. Loop over each selected individual, ask the question, and map the answer
responses = []
for _, row in ensu_sel.iterrows():
    system_prompt = f"Tu eres {row['SEXO']}-que tiene {row['EDAD']}- años de edad y originario de {row['NOM_MUN']}."
    question = survey_questions["BP1_1"].format(NOM_MUN=row["NOM_MUN"])

    # Initialize model
    model = genai.GenerativeModel(
        "gemini-2.5-flash-preview-05-20",
        #"gemini-1.5-pro",
        #"gemini-1.5-flash",
        #"gemini-2.0-flash",
        system_instruction=system_prompt
    )

    # Generate content
    response = model.generate_content(
        f"Responde esta pregunta as JSON: {{\"response_code\": [1, 2, or 3]}}.\n\nQuestion: {question}",
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema=SurveyResponse
        )
    )

    # Parse response
    try:
        resp_json = json.loads(response.text)
        code = resp_json.get("response_code", None)
    except Exception as e:
        code = None  # In case of failure, fallback to None

    label = BP1_dic.get(code, "Unknown")

    responses.append({
      "NOM_MUN": row["NOM_MUN"],
      "SEXO": row["SEXO"],
      "EDAD": row["EDAD"],
      "response_code": code,
      "response_label": label
  })

# Create results DataFrame
results_df = pd.DataFrame(responses)
print(results_df)


              NOM_MUN    SEXO  EDAD  response_code         response_label
0  ACAPULCO DE JUAREZ   Mujer    76              3  No sabe / no responde
1      AGUASCALIENTES  Hombre    39              2               Inseguro
2      AGUASCALIENTES  Hombre    19              1                 Seguro
3    TUXTLA GUTIERREZ  Hombre    42              2               Inseguro
4           QUERETARO  Hombre    53              2               Inseguro
5          MANZANILLO   Mujer    51              1                 Seguro
6           CHIHUAHUA   Mujer    58              2               Inseguro
7           GUADALUPE  Hombre    74              2               Inseguro
8              MERIDA  Hombre    59              1                 Seguro
9                LEON   Mujer    20              2               Inseguro


### **We create a function for calculating the accuracy and the error in the LLM models**

In [21]:
from sklearn.metrics import confusion_matrix
import pandas as pd

def evaluate_llm_responses(results_df: pd.DataFrame, ensu_sample: pd.DataFrame):
    """
    Join on NOM_MUN, SEXO, and EDAD, then compare response_code with BP1_1.
    Computes accuracy and confusion matrix.

    Returns:
        accuracy (float): % of correct matches
        conf_matrix (pd.DataFrame): Confusion matrix as DataFrame
    """
    # Merge based on key identifiers
    merged = pd.merge(
        results_df,
        ensu_sample[["NOM_MUN", "SEXO", "EDAD", "BP1_1"]],
        on=["NOM_MUN", "SEXO", "EDAD"],
        how="inner"
    )

    # Drop rows where predictions or true labels are missing
    merged = merged.dropna(subset=["response_code", "BP1_1"])

    # Accuracy
    correct = (merged["response_code"] == merged["BP1_1"]).sum()
    total = len(merged)
    accuracy = (correct / total) * 100 if total > 0 else 0.0

    # Confusion matrix
    labels = sorted(merged["BP1_1"].dropna().unique())
    cm = confusion_matrix(
        merged["BP1_1"], merged["response_code"], labels=labels
    )
    cm_df = pd.DataFrame(cm, index=[f"True_{i}" for i in labels], columns=[f"Pred_{i}" for i in labels])

    return accuracy, cm_df

In [26]:
accuracy, conf_matrix = evaluate_llm_responses(results_df, ensu_sample)

print(f"Accuracy: {accuracy:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 60.00%
Confusion Matrix:
        Pred_1  Pred_2
True_1       1       1
True_2       2       5


In [27]:
### EXAMPLE 2.  We are using a more descriptive prompt
import pandas as pd
import json
from typing_extensions import TypedDict
import google.generativeai as genai

# 1. Assume ensu_sample is already loaded as a pandas DataFrame.

# Select only the variables city, sex, and age
ensu_sel = ensu_sample[["NOM_MUN", "SEXO", "EDAD"]]

# 2. Define the survey question template
survey_questions = {
    "BP1_1": "En términos de delincuencia, ¿considera que vivir actualmente en {NOM_MUN} es…"
}

#3. Define the response mapping dictionary
BP1_dic = {
    1: "Seguro?",
    2: "Inseguro?",
    3: "No sabe / no responde"
}

# 4. Define expected JSON output schema
class SurveyResponse(TypedDict):
    response_code: int

# 5. Loop over each selected individual, ask the question, and map the answer
responses = []
for _, row in ensu_sel.iterrows():
    system_prompt = (
    f"Eres una persona de sexo {row['SEXO']}, tienes {row['EDAD']} años, y vives en el municipio de {row['NOM_MUN']}. "
    f"Conoces bien la situación de seguridad pública y crimen en {row['NOM_MUN']}, ya que has leído noticias, has visto información en redes sociales, "
    f"y has conversado con tus vecinos. Basa tus respuestas en tu conocimiento sobre la situación seguridad pública  en {row['NOM_MUN']}."
    f"Manten tus respestas ecuánimes y objetivas "
    )

    question = survey_questions["BP1_1"].format(NOM_MUN=row["NOM_MUN"])

    # Initialize model
    model = genai.GenerativeModel(
        "gemini-2.5-flash-preview-05-20",
        #"gemini-1.5-pro",
        #"gemini-1.5-flash",
        #"gemini-2.0-flash",
        system_instruction=system_prompt
    )

    # Generate content
    prompt = f"""
    Responde esta pregunta como un número JSON válido según las siguientes reglas:

    1: "Seguro?" — si te sientes seguro o segura cuando vives en {row['NOM_MUN']}
    2: "Inseguro?" — si te sientes inseguro o insegura cuando vive en {row['NOM_MUN']}
    3: "No sabe / no responde" — si no sabes o no puedes responder

    Devuelve solo el siguiente formato JSON:
    {{"response_code": 1}}, {{"response_code": 2}}, o {{"response_code": 3}}

    Pregunta: {question}
    """

    response = model.generate_content(
        prompt.strip(),
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema=SurveyResponse
        )
    )

    # Parse response
    try:
        resp_json = json.loads(response.text)
        code = resp_json.get("response_code", None)
    except Exception as e:
        code = None  # In case of failure, fallback to None

    label = BP1_dic.get(code, "Unknown")

    responses.append({
      "NOM_MUN": row["NOM_MUN"],
      "SEXO": row["SEXO"],
      "EDAD": row["EDAD"],
      "response_code": code,
      "response_label": label
  })

# Create results DataFrame
results_df = pd.DataFrame(responses)
print(results_df)


              NOM_MUN    SEXO  EDAD  response_code response_label
0  ACAPULCO DE JUAREZ   Mujer    76              2      Inseguro?
1      AGUASCALIENTES  Hombre    39              2      Inseguro?
2      AGUASCALIENTES  Hombre    19              2      Inseguro?
3    TUXTLA GUTIERREZ  Hombre    42              2      Inseguro?
4           QUERETARO  Hombre    53              1        Seguro?
5          MANZANILLO   Mujer    51              2      Inseguro?
6           CHIHUAHUA   Mujer    58              2      Inseguro?
7           GUADALUPE  Hombre    74              2      Inseguro?
8              MERIDA  Hombre    59              1        Seguro?
9                LEON   Mujer    20              2      Inseguro?


In [28]:
accuracy, conf_matrix = evaluate_llm_responses(results_df, ensu_sample)

print(f"Accuracy: {accuracy:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 50.00%
Confusion Matrix:
        Pred_1  Pred_2
True_1       0       3
True_2       2       5
